In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import pandas as pd 

#QA
inputs = [
    "What is the name of the first chapter in this pdf",
    "what is the atomic number of hydrogen?",
    "Total number of chapters in this document?",
]

outputs = [
    "The first chapter is about Physical Quantities and Measurement.",
    "The atomic number of hydrogen is 1.",
    "There are a total of 16 chapters in this document."
]

# dataset
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
df = pd.DataFrame(qa_pairs)

# write to csv
csv_path = "../data/goldens.csv"
df.to_csv(csv_path, index=False)



In [3]:
"""from langsmith import Client 

client = Client()
dataset_name = "llmops_dataset_new0"

# Store 
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="input and expected output pairs for llmops series",
)

client.create_examples(
    inputs = [{"question": q} for q in inputs],
    outputs = [{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)"""

'from langsmith import Client \n\nclient = Client()\ndataset_name = "llmops_dataset_new0"\n\n# Store \ndataset = client.create_dataset(\n    dataset_name=dataset_name,\n    description="input and expected output pairs for llmops series",\n)\n\nclient.create_examples(\n    inputs = [{"question": q} for q in inputs],\n    outputs = [{"answer": a} for a in outputs],\n    dataset_id=dataset.id,\n)'

In [4]:
import sys
sys.path.append("D:\\Machine learning\\llmops_series")

from pathlib import Path
from src.document_ingestion.data_ingestion import ChatIngestor
from src.document_chat.retrieval import ConversationalRAG 
import os


# Simple file adapter for local file paths
class LocalFileAdapter:
    """Adapter for local file paths to work with ChatIngestor."""
    def __init__(self, file_path: str):
        self.path = Path(file_path)
        self.name = self.path.name
        self.filename = self.path.name  # Add filename attribute for compatibility
    
    def read(self) -> bytes:
        """Read method for file-like interface."""
        return self.path.read_bytes()
    
    def getbuffer(self) -> bytes:
        """Get buffer method for compatibility."""
        return self.path.read_bytes()


def answer_ai_report_question(
    inputs: dict,
    data_path: str = "D://Machine learning/llmops_series/data/Physics 9th Ch 1 Final.pdf",
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    k: int = 5
) -> dict:
    """
    Answer questions about the AI Engineering Report using RAG.
    
    Args:
        inputs: Dictionary containing the question, e.g., {"question": "What is RAG?"}
        data_path: Path to the AI Engineering Report text file
        chunk_size: Size of text chunks for splitting
        chunk_overlap: Overlap between chunks
        k: Number of documents to retrieve
    
    Returns:
        Dictionary with the answer, e.g., {"answer": "RAG stands for..."}
    """
    try:
        # Extract question from inputs
        question = inputs.get("question", "")
        if not question:
            return {"answer": "No question provided"}
        
        # Check if file exists
        if not Path(data_path).exists():
            return {"answer": f"Data file not found: {data_path}"}
        
        # Create file adapter
        file_adapter = LocalFileAdapter(data_path)
        
        # Build index using ChatIngestor
        ingestor = ChatIngestor(
            temp_base="data",
            faiss_base="faiss_index",
            use_session_dirs=True
        )
        
        # Build retriever
        ingestor.built_retriever(
            uploaded_files=[file_adapter],
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            k=k
        )
        
        # Get session ID and index path
        session_id = ingestor.session_id
        index_path = f"faiss_index/{session_id}"
        
        # Create RAG instance and load retriever
        rag = ConversationalRAG(session_id=session_id)
        rag.load_retriever_from_faiss(
            index_path=index_path,
            k=k,
            index_name=os.getenv("FAISS_INDEX_NAME", "index")
        )
        
        # Get answer
        answer = rag.invoke(question, chat_history=[])
        
        return {"answer": answer}
        
    except Exception as e:
        return {"answer": f"Error: {str(e)}"}

In [5]:
# Test the function 
test_input = {"question": "What is the name of the first chapter in this pdf?"}
result = answer_ai_report_question(test_input)
print("question:", test_input["question"])
print("answer:", result["answer"])

{"timestamp": "2026-01-17T05:45:42.063824Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-17T05:45:42.064839Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-17T05:45:42.067923Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_Kv...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2026-01-17T05:45:42.069446Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm"], "timestamp": "2026-01-17T05:45:42.085522Z", "level": "info", "event": "YAML config loaded"}
{"session_id": "session_20260117_104542_551b3de3", "temp_dir": "data\\session_20260117_104542_551b3de3", "faiss_dir": "faiss_index\\session_20260117_104542_551b3de3", "sessionized": true, "timestamp": "2026-01-17T05:45:42.087291Z", "level": "info", "event": "ChatIngestor initialized"}
{"uploaded": "Physics 9th Ch 1 Final.pdf", "saved_as": "da

question: What is the name of the first chapter in this pdf?
answer: Error: Error in [d:\Machine learning\llmops_series\.venv\Lib\site-packages\google\api_core\grpc_helpers.py] at line [77] | Message: Invocation error in ConversationalRAG
Traceback:
Traceback (most recent call last):
  File "D:\Machine learning\llmops_series\src\document_chat\retrieval.py", line 129, in invoke
    answer = self.chain.invoke(payload)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Machine learning\llmops_series\.venv\Lib\site-packages\langchain_core\runnables\base.py", line 3044, in invoke
    input_ = context.run(step.invoke, input_, config, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Machine learning\llmops_series\.venv\Lib\site-packages\langchain_core\runnables\base.py", line 3773, in invoke
    output = {key: future.result() for key, future in zip(steps, futures)}
                   ^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\concurrent\futures\

In [6]:
# Example : Test with all golden questions
print("Testing all questions from the dataset:\n")
for i, q in enumerate(inputs, 1):
    test_input = {"question": q}
    result = answer_ai_report_question(test_input)
    print(f"Q{i}: {q}")
    print(f"A{i}: {result['answer']}\n")
    print("-" * 80 + "\n")    

{"timestamp": "2026-01-17T05:45:50.188612Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-17T05:45:50.188612Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-17T05:45:50.188612Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_Kv...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2026-01-17T05:45:50.188612Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm"], "timestamp": "2026-01-17T05:45:50.200751Z", "level": "info", "event": "YAML config loaded"}
{"session_id": "session_20260117_104550_700def14", "temp_dir": "data\\session_20260117_104550_700def14", "faiss_dir": "faiss_index\\session_20260117_104550_700def14", "sessionized": true, "timestamp": "2026-01-17T05:45:50.203452Z", "level": "info", "event": "ChatIngestor initialized"}
{"uploaded": "Physics 9th Ch 1 Final.pdf", "saved_as": "da

Testing all questions from the dataset:



{"count": 23, "timestamp": "2026-01-17T05:45:51.551109Z", "level": "info", "event": "Documents loaded"}
{"chunks": 62, "chunk_size": 1000, "overlap": 200, "timestamp": "2026-01-17T05:45:51.559629Z", "level": "info", "event": "Documents split"}
{"model": "models/text-embedding-004", "timestamp": "2026-01-17T05:45:51.564976Z", "level": "info", "event": "Loading embedding model"}
{"added": 1, "index": "faiss_index\\session_20260117_104550_700def14", "timestamp": "2026-01-17T05:45:55.245595Z", "level": "info", "event": "FAISS index updated"}
{"k": 5, "fetch_k": 20, "lambda_mult": 0.5, "timestamp": "2026-01-17T05:45:55.245595Z", "level": "info", "event": "Using MMR search"}
{"timestamp": "2026-01-17T05:45:55.250914Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-17T05:45:55.250914Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-17T05:45:55.250914Z", "level": "info", "event": "Loaded GOOGLE_API_KEY f

Q1: What is the name of the first chapter in this pdf
A1: Error: Error in [d:\Machine learning\llmops_series\.venv\Lib\site-packages\google\api_core\grpc_helpers.py] at line [77] | Message: Invocation error in ConversationalRAG
Traceback:
Traceback (most recent call last):
  File "D:\Machine learning\llmops_series\src\document_chat\retrieval.py", line 129, in invoke
    answer = self.chain.invoke(payload)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Machine learning\llmops_series\.venv\Lib\site-packages\langchain_core\runnables\base.py", line 3044, in invoke
    input_ = context.run(step.invoke, input_, config, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Machine learning\llmops_series\.venv\Lib\site-packages\langchain_core\runnables\base.py", line 3773, in invoke
    output = {key: future.result() for key, future in zip(steps, futures)}
                   ^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\concurrent\futures\_base.py", 

{"count": 23, "timestamp": "2026-01-17T05:45:59.169716Z", "level": "info", "event": "Documents loaded"}
{"chunks": 62, "chunk_size": 1000, "overlap": 200, "timestamp": "2026-01-17T05:45:59.187229Z", "level": "info", "event": "Documents split"}
{"model": "models/text-embedding-004", "timestamp": "2026-01-17T05:45:59.187690Z", "level": "info", "event": "Loading embedding model"}
{"added": 1, "index": "faiss_index\\session_20260117_104558_23620208", "timestamp": "2026-01-17T05:46:02.361059Z", "level": "info", "event": "FAISS index updated"}
{"k": 5, "fetch_k": 20, "lambda_mult": 0.5, "timestamp": "2026-01-17T05:46:02.364379Z", "level": "info", "event": "Using MMR search"}
{"timestamp": "2026-01-17T05:46:02.368577Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-17T05:46:02.372745Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-17T05:46:02.372745Z", "level": "info", "event": "Loaded GOOGLE_API_KEY f

Q2: what is the atomic number of hydrogen?
A2: Error: Error in [d:\Machine learning\llmops_series\.venv\Lib\site-packages\google\api_core\grpc_helpers.py] at line [77] | Message: Invocation error in ConversationalRAG
Traceback:
Traceback (most recent call last):
  File "D:\Machine learning\llmops_series\src\document_chat\retrieval.py", line 129, in invoke
    answer = self.chain.invoke(payload)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Machine learning\llmops_series\.venv\Lib\site-packages\langchain_core\runnables\base.py", line 3044, in invoke
    input_ = context.run(step.invoke, input_, config, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Machine learning\llmops_series\.venv\Lib\site-packages\langchain_core\runnables\base.py", line 3773, in invoke
    output = {key: future.result() for key, future in zip(steps, futures)}
                   ^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\concurrent\futures\_base.py", line 456, i

{"count": 23, "timestamp": "2026-01-17T05:46:06.970075Z", "level": "info", "event": "Documents loaded"}
{"chunks": 62, "chunk_size": 1000, "overlap": 200, "timestamp": "2026-01-17T05:46:06.973863Z", "level": "info", "event": "Documents split"}
{"model": "models/text-embedding-004", "timestamp": "2026-01-17T05:46:06.978544Z", "level": "info", "event": "Loading embedding model"}
{"added": 1, "index": "faiss_index\\session_20260117_104605_98c202cb", "timestamp": "2026-01-17T05:46:10.087621Z", "level": "info", "event": "FAISS index updated"}
{"k": 5, "fetch_k": 20, "lambda_mult": 0.5, "timestamp": "2026-01-17T05:46:10.089921Z", "level": "info", "event": "Using MMR search"}
{"timestamp": "2026-01-17T05:46:10.099870Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-17T05:46:10.104282Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-17T05:46:10.107892Z", "level": "info", "event": "Loaded GOOGLE_API_KEY f

Q3: Total number of chapters in this document?
A3: Error: Error in [d:\Machine learning\llmops_series\.venv\Lib\site-packages\google\api_core\grpc_helpers.py] at line [77] | Message: Invocation error in ConversationalRAG
Traceback:
Traceback (most recent call last):
  File "D:\Machine learning\llmops_series\src\document_chat\retrieval.py", line 129, in invoke
    answer = self.chain.invoke(payload)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Machine learning\llmops_series\.venv\Lib\site-packages\langchain_core\runnables\base.py", line 3044, in invoke
    input_ = context.run(step.invoke, input_, config, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Machine learning\llmops_series\.venv\Lib\site-packages\langchain_core\runnables\base.py", line 3773, in invoke
    output = {key: future.result() for key, future in zip(steps, futures)}
                   ^^^^^^^^^^^^^^^
  File "C:\Program Files\Python312\Lib\concurrent\futures\_base.py", line 45

In [7]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

qa_evaluator = [LangChainStringEvaluator("cot_qa")]
dataset_name = "llmops_dataset_new"

# Run evaluation using our RAG function
experiment_results = evaluate(
    answer_ai_report_question,
    data=dataset_name,
    evaluators=qa_evaluator,
    experiment_prefix="test-llmops_dataset-qa-rag",
    metadata={
        "variant": "RAG with FAISS and AI Engineering Report",
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "k": 5
    },
)

d:\Machine learning\llmops_series\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'test-llmops_dataset-qa-rag-32385ac8' at:
https://smith.langchain.com/o/e8008ce9-eb06-5861-b8e8-97dd6daeac49/datasets/60a4e1ba-caf7-4cd7-9450-04c89b74c7fa/compare?selectedSessions=5e3e15ad-7db9-4d9b-8cc8-8e9ed1d3e8b1




0it [00:00, ?it/s]{"timestamp": "2026-01-17T05:46:19.033968Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-17T05:46:19.033968Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-17T05:46:19.033968Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_Kv...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2026-01-17T05:46:19.043127Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm"], "timestamp": "2026-01-17T05:46:19.044569Z", "level": "info", "event": "YAML config loaded"}
{"session_id": "session_20260117_104619_0ead42a2", "temp_dir": "data\\session_20260117_104619_0ead42a2", "faiss_dir": "faiss_index\\session_20260117_104619_0ead42a2", "sessionized": true, "timestamp": "2026-01-17T05:46:19.049859Z", "level": "info", "event": "ChatIngestor initialized"}
{"uploaded": "Physics 9th Ch 1 Final.pdf